In [33]:
import pdfplumber
import pandas as pd
import numpy as np

In [34]:
# define the columns as a list
# the two pages of the table are parsing with different number of columns, so I'm building 2 separate dfs
columns = ['col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'col7', 'col8']
columns2 = ['col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'col7', 'col8', 'col9', 'col10', 'col11', 'col12', 'col13']

# where does the PDF live?
pdf_file = 'KS2016.pdf'

# create an empty dataframe
df = pd.DataFrame(columns=columns)
df2 = pd.DataFrame(columns=columns2)

In [35]:
# define function for first page
def page_to_df(page):
    
    # find the table on the page and extract the data
    table = page.extract_table(table_settings={"vertical_strategy": "text"})
    
    # grab all rows in the table 
    # including first one, because our table_settings mean it already ignores header row
    lines = table
    
    # return the data in a data frame
    return pd.DataFrame(lines, columns=columns)

In [36]:
# define function for second page
def page_to_df2(page):
    
    # find the table on the page and extract the data
    table = page.extract_table(table_settings={"vertical_strategy": "text"})
    
    # grab all rows in the table 
    # including first one, because our table_settings mean it already ignores header row
    lines = table
    
    # return the data in a data frame
    return pd.DataFrame(lines, columns=columns2)

In [37]:
# extract the first page
with pdfplumber.open(pdf_file) as pdf:
    pg1 = pdf.pages[0]
    df1 = page_to_df(pg1)

In [38]:
# check out results of first page
df1.head()

,col1,col2,col3,col4,col5,col6,col7,col8
0,Allen,15.2%,"2 ,995",22.9%,690,78%,,22%
1,Anderson,14.7%,"1 ,972",22.8%,450,70%,,30%
2,Atchison,16.9%,"3 ,880",24.4%,950,69%,,31%
3,Barber,13.7%,"1 ,108",21.2%,230,84%,,16%
4,Barton,12.8%,"6 ,637",21.3%,"1,410",78%,,22%


In [39]:
# extract the second page
with pdfplumber.open(pdf_file) as pdf:
    pg2 = pdf.pages[1]
    df2 = page_to_df2(pg2)

In [40]:
# check out results of second page
df2.head()

,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13
0,Mitchell,,12.6%,,"1 ,405",,19.4%,,270,74%,,26%,
1,Montgomery,,16.5%,,"7 ,980",,24.2%,,"1,930",82%,,18%,
2,Morris,,11.3%,,"1 ,182",,18.2%,,220,69%,,32%,
3,Morton,,9.9%,,6 58,,17.3%,,110,56%,,44%,
4,Nemaha,,12.0%,,"2 ,642",,18.1%,,480,57%,,43%,


In [41]:
# remove empty columns in each df
# df1
df1 = df1.drop(['col7'], axis=1)
df1.head()

,col1,col2,col3,col4,col5,col6,col8
0,Allen,15.2%,"2 ,995",22.9%,690,78%,22%
1,Anderson,14.7%,"1 ,972",22.8%,450,70%,30%
2,Atchison,16.9%,"3 ,880",24.4%,950,69%,31%
3,Barber,13.7%,"1 ,108",21.2%,230,84%,16%
4,Barton,12.8%,"6 ,637",21.3%,"1,410",78%,22%


In [44]:
# change col names so we can merge
df1 = df1.rename(index=str, columns={"col1": "county", "col2": "insecure_rate", "col3": "child_pop", 
                                    "col4": "child_insecure_rate", "col5": "child_insecure_pop", 
                                     "col6": "insecure_eligible", "col8": "insecure_ineligible"})

# fix bug in import of Johnson County
df1.at['45', 'insecure_rate'] = '10.6%'
df1.at['45', 'child_pop'] = '145,152'

pd.options.display.height = 225
df1

height has been deprecated.



,county,insecure_rate,child_pop,child_insecure_rate,child_insecure_pop,insecure_eligible,insecure_ineligible
0,Allen,15.2%,"2 ,995",22.9%,690,78%,22%
1,Anderson,14.7%,"1 ,972",22.8%,450,70%,30%
2,Atchison,16.9%,"3 ,880",24.4%,950,69%,31%
3,Barber,13.7%,"1 ,108",21.2%,230,84%,16%
4,Barton,12.8%,"6 ,637",21.3%,"1,410",78%,22%
5,Bourbon,15.8%,"3 ,719",24.0%,890,87%,13%
6,Brown,13.8%,"2 ,458",21.3%,520,77%,24%
7,Butler,12.2%,"1 7,220",18.7%,"3,230",67%,33%
8,Chase,12.4%,6 07,19.7%,120,72%,28%
9,Chautauqua,14.2%,7 09,22.8%,160,96%,4%


In [45]:
# df2
df2 = df2.drop(['col2', 'col4', 'col6', 'col8', 'col11', 'col13'], axis=1)
df2.head()

,col1,col3,col5,col7,col9,col10,col12
0,Mitchell,12.6%,"1 ,405",19.4%,270,74%,26%
1,Montgomery,16.5%,"7 ,980",24.2%,"1,930",82%,18%
2,Morris,11.3%,"1 ,182",18.2%,220,69%,32%
3,Morton,9.9%,6 58,17.3%,110,56%,44%
4,Nemaha,12.0%,"2 ,642",18.1%,480,57%,43%


In [46]:
# change col names so we can merge
df2 = df2.rename(index=str, columns={"col1": "county", "col3": "insecure_rate", "col5": "child_pop", 
                                    "col7": "child_insecure_rate", "col9": "child_insecure_pop", 
                                     "col10": "insecure_eligible", "col12": "insecure_ineligible"})
df2.head()

,county,insecure_rate,child_pop,child_insecure_rate,child_insecure_pop,insecure_eligible,insecure_ineligible
0,Mitchell,12.6%,"1 ,405",19.4%,270,74%,26%
1,Montgomery,16.5%,"7 ,980",24.2%,"1,930",82%,18%
2,Morris,11.3%,"1 ,182",18.2%,220,69%,32%
3,Morton,9.9%,6 58,17.3%,110,56%,44%
4,Nemaha,12.0%,"2 ,642",18.1%,480,57%,43%


In [47]:
dataKS = pd.concat([df1,df2])
dataKS.info()

<class 'pandas.core.frame.DataFrame'>
Index: 106 entries, 0 to 44
Data columns (total 7 columns):
county                 106 non-null object
insecure_rate          106 non-null object
child_pop              106 non-null object
child_insecure_rate    106 non-null object
child_insecure_pop     106 non-null object
insecure_eligible      106 non-null object
insecure_ineligible    106 non-null object
dtypes: object(7)
memory usage: 6.6+ KB


In [48]:
# DATA CLEANING

# strip middle whitespace
dataKS['child_pop'] = dataKS['child_pop'].str.strip().str.lower().str.replace(' ', '')

# and trim any other leading/trailing spaces

# remove percent signs
dataKS['insecure_rate'] = dataKS['insecure_rate'].str.strip().str.lower().str.replace('%', '')
dataKS['child_insecure_rate'] = dataKS['child_insecure_rate'].str.strip().str.lower().str.replace('%', '')
dataKS['insecure_eligible'] = dataKS['insecure_eligible'].str.strip().str.lower().str.replace('%', '')
dataKS['insecure_ineligible'] = dataKS['insecure_ineligible'].str.strip().str.lower().str.replace('%', '')


In [49]:
# turn percents into numbers
dataKS['insecure_rate'] = pd.to_numeric(dataKS['insecure_rate'], errors='coerce')
dataKS['child_insecure_rate'] = pd.to_numeric(dataKS['child_insecure_rate'], errors='coerce')
dataKS['insecure_eligible'] = pd.to_numeric(dataKS['insecure_eligible'], errors='coerce')
dataKS['insecure_ineligible'] = pd.to_numeric(dataKS['insecure_ineligible'], errors='coerce')

# turn percents into decimals
dataKS['insecure_rate'] = (dataKS['insecure_rate'] / 100)
dataKS['child_insecure_rate'] = (dataKS['child_insecure_rate'] / 100)
dataKS['insecure_eligible'] = (dataKS['insecure_eligible'] / 100)
dataKS['insecure_ineligible'] = (dataKS['insecure_ineligible'] / 100)

In [50]:
dataKS.head()

,county,insecure_rate,child_pop,child_insecure_rate,child_insecure_pop,insecure_eligible,insecure_ineligible
0,Allen,0.152,"2,995",0.229,690,0.78,0.22
1,Anderson,0.147,"1,972",0.228,450,0.70,0.30
2,Atchison,0.169,"3,880",0.244,950,0.69,0.31
3,Barber,0.137,"1,108",0.212,230,0.84,0.16
4,Barton,0.128,"6,637",0.213,"1,410",0.78,0.22


In [51]:
# reset index
dataKS = dataKS.reset_index()

In [52]:
# remove last row containing summary data
dataKS = dataKS.drop(105)

In [53]:
# check end of file
dataKS.tail()

,index,county,insecure_rate,child_pop,child_insecure_rate,child_insecure_pop,insecure_eligible,insecure_ineligible
100,39,Washington,0.117,"1,286",0.193,250,0.67,0.33
101,40,Wichita,0.049,630,0.127,80,1.00,0.00
102,41,Wilson,0.152,"2,133",0.246,530,0.85,0.15
103,42,Woodson,0.166,679,0.264,180,0.79,0.21
104,43,Wyandotte,0.168,"45,625",0.237,"10,810",0.74,0.26


In [54]:
# write to csv
dataKS.to_csv('foodinsecurity_KS.csv')